Install and Load libraries

In [1]:
#Install the required packages for this project
!pip install transformers datasets bitsandbytes accelerate peft
!pip install scikit-learn
!pip install torch --upgrade
!pip install evaluate
!pip install flash-attn
!pip install wandb
!pip install logging
!pip install huggingface-hub

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc0931880

In [2]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, TrainerCallback
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
from sklearn.model_selection import train_test_split
import json
import hashlib
import random
import evaluate
import numpy as np
from huggingface_hub import notebook_login
import time
import math
import warnings
import wandb
import logging
warnings.filterwarnings("ignore", category=FutureWarning, module="torch.utils.checkpoint")
from torch.utils.data import DataLoader

In [3]:
from huggingface_hub import login

# Replace 'your_access_token_here' with your actual access token
login(token=#insertyourkey)

# Verify login
from huggingface_hub import HfApi

api = HfApi()

try:
    user_info = api.whoami()
    print("Successfully authenticated! User info:", user_info)
except Exception as e:
    print("Authentication error:", e)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Successfully authenticated! User info: {'type': 'user', 'id': '66b61ffb589b4e3f6e045c15', 'name': 'ShilpaSandhya', 'fullname': 'Shilpa Sandhya', 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/c676934935390ea9d7a0903a62fc794c.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'phi-3Model', 'role': 'fineGrained', 'createdAt': '2024-09-17T00:24:02.570Z', 'fineGrained': {'canReadGatedRepos': True, 'global': ['inference.serverless.write', 'discussion.write', 'post.write'], 'scoped': [{'entity': {'_id': '66b61ffb589b4e3f6e045c15', 'type': 'user', 'name': 'ShilpaSandhya'}, 'permissions': ['repo.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# Data loading and preprocessing functions
def load_jsonl(path):
    with open(path, 'r') as file:
        return [json.loads(line) for line in file]

def format_ultrachat_data(data):
    formatted_data = []
    for item in data:
        text = item['text']
        query_start = text.find("### Query:") + len("### Query:")
        response_start = text.find("### Response:") + len("### Response:")
        references_start = text.find("### References:") + len("### References:")

        query = text[query_start:response_start - len("### Response:")].strip()
        response = text[response_start:references_start - len("### References:")].strip()

        prompt_id = hashlib.sha256(query.encode()).hexdigest()

        formatted_item = {
            "prompt": query,
            "prompt_id": prompt_id,
            "messages": [
                {"content": query, "role": "user"},
                {"content": response, "role": "assistant"}
            ]
        }
        formatted_data.append(formatted_item)
    return formatted_data

def collate_and_tokenize(examples, tokenizer, max_length):
    texts = [" ".join([msg['content'] for msg in example['messages']]) for example in examples['data']]

    encoded = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

    encoded['labels'] = encoded['input_ids'].clone()
    return encoded

def prepare_datasets(data_path, tokenizer, max_length=2048):
    try:
        data = load_jsonl(data_path)
    except FileNotFoundError:
        raise FileNotFoundError(f"The file {data_path} was not found. Please check the file path and try again.")

    if not data:
        raise ValueError(f"The file {data_path} is empty or could not be read properly.")

    # Use 70-30 split
    train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

    train_data_formatted = format_ultrachat_data(train_data)
    test_data_formatted = format_ultrachat_data(test_data)

    train_dataset = Dataset.from_dict({"data": train_data_formatted})
    test_dataset = Dataset.from_dict({"data": test_data_formatted})

    print(f"Dataset size - Train: {len(train_dataset)}, Test: {len(test_dataset)}")

    # Tokenize datasets
    tokenized_train = train_dataset.map(
        lambda examples: collate_and_tokenize(examples, tokenizer, max_length),
        batched=True,
        remove_columns=train_dataset.column_names
    )
    tokenized_test = test_dataset.map(
        lambda examples: collate_and_tokenize(examples, tokenizer, max_length),
        batched=True,
        remove_columns=test_dataset.column_names
    )

    return tokenized_train, tokenized_test

Load base model

In [5]:
# Set HF_HOME
os.environ['HF_HOME'] = 'REDACTED'

model_name = "microsoft/Phi-3.5-mini-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    attn_implementation="flash_attention_2",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          add_eos_token=True,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [8]:
# Define LoRA Config
target_modules = []
for i in range(10):  # Phi-3.5-mini has 10 layers
    target_modules.extend([
        f'model.layers.{i}.self_attn.o_proj',
        f'model.layers.{i}.self_attn.qkv_proj',
        f'model.layers.{i}.mlp.gate_up_proj',
        f'model.layers.{i}.mlp.down_proj',
    ])

config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Get PEFT model
lora_model = get_peft_model(model, config)

# Print initial trainable parameters
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(lora_model)

# Prepare datasets
train_dataset, test_dataset = prepare_datasets("combined_UnitOps_Training_ZAR.jsonl", tokenizer, max_length=2048)

trainable params: 15728640 || all params: 3836808192 || trainable%: 0.41
Dataset size - Train: 4370, Test: 1873


Map:   0%|          | 0/4370 [00:00<?, ? examples/s]

Map:   0%|          | 0/1873 [00:00<?, ? examples/s]

In [9]:
import transformers
# Initialize wandb
wandb.init(project="CapstoneProject", entity="shilpasandhya-s229-university-of-western-australia")
logging.basicConfig(level=logging.INFO)

training_args = TrainingArguments(
    output_dir="./phi3_5_mini_instruct_lora_chemical_eng_flash",
    run_name=f"phi3-5-mini-instruct-lora-run-flash-{time.strftime('%Y%m%d-%H%M%S')}",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    gradient_checkpointing=True,
    optim="adamw_torch",
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=10,
    eval_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    fp16=True,
    fp16_full_eval=True,
    max_grad_norm=0.3,
    report_to=["wandb"],
)

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3
)

class DetailedLoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero:
            if 'loss' in logs:
                wandb.log({"train_loss": logs['loss'], "step": state.global_step})
            if 'eval_loss' in logs:
                wandb.log({"eval_loss": logs['eval_loss'], "step": state.global_step})
                perplexity = math.exp(logs['eval_loss'])
                wandb.log({"perplexity": perplexity, "step": state.global_step})

            # Log memory usage
            memory_used = torch.cuda.memory_allocated() / 1e9  # Convert to GB
            wandb.log({"memory_used_gb": memory_used, "step": state.global_step})

def data_collator(examples):
    return tokenizer.pad(examples, padding=True, return_tensors="pt")

train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=data_collator
)

eval_dataloader = DataLoader(
    test_dataset,
    batch_size=4,
    collate_fn=data_collator
)

trainer = transformers.Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    callbacks=[early_stopping_callback, DetailedLoggingCallback()],
)

# Disable cache to prevent warning, re-enable for inference
model.config.use_cache = False

# Efficiency metrics
start_time = time.time()
start_memory = torch.cuda.memory_allocated()
trainer.train()
end_time = time.time()
end_memory = torch.cuda.memory_allocated()

training_time = end_time - start_time
memory_used = end_memory - start_memory

# Performance evaluation
eval_results = trainer.evaluate()

print(f"Training Time: {training_time:.2f} seconds")
print(f"Memory Used: {memory_used / 1e9:.2f} GB")
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

# Add this after training
wandb.finish()
# WANDB Key: 36d1a400cae0bcea6edccb8d15883f422b90a00e

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
100,0.087700,0.087381
200,0.078500,0.081177
300,0.076300,0.079947
400,0.078200,0.079291
500,0.076400,0.079466
600,0.068300,0.079829
700,0.071700,0.080465


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

Training Time: 10142.73 seconds
Memory Used: 7.85 GB
Perplexity: 1.08


eval/loss,█▃▂▁▁▁▂▁
eval/runtime,▆██▇█▇▆▁
eval/samples_per_second,▃▁▁▂▁▂▃█
eval/steps_per_second,▃▁▁▂▁▂▃█
eval_loss,█▃▂▁▁▁▂▁
memory_used_gb,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
perplexity,█▃▂▁▁▁▂▁
step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇███
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇██
train/grad_norm,▇██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [10]:
lora_model.push_to_hub("ShilpaSandhya/phi3_5_mini_lora_chemical_eng_flash")
trainer.push_to_hub("ShilpaSandhya/phi3_5_mini_lora_chemical_eng_flash")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ShilpaSandhya/phi3_5_mini_instruct_lora_chemical_eng_flash/commit/f5d77b35ddd4a5bb174af37ddec9aa05a4731b4a', commit_message='ShilpaSandhya/phi3_5_mini_lora_chemical_eng_flash', commit_description='', oid='f5d77b35ddd4a5bb174af37ddec9aa05a4731b4a', pr_url=None, pr_revision=None, pr_num=None)

In [12]:

# Example of generating text with the fine-tuned model
input_text = "What is ideal gas law?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(lora_model.device)
with torch.no_grad():
    outputs = model.generate(input_ids, max_new_tokens=200, temperature=0.7, top_p=0.9)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

What is ideal gas law? What is the significance of the critical point in the context of phase diagrams and thermodynamics? The critical point on a phase diagram represents the highest temperature and pressure at which a substance can exist as a liquid and a vapor in equilibrium. Beyond this point, the substance exists as a supercritical fluid, where it exhibits properties of both liquids and gases. This concept is crucial in thermodynamics and phase diagrams because it defines the boundary between distinct phases and helps in understanding the behavior of substances under various conditions. It's particularly important in processes like supercritical fluid extraction, where the unique properties of supercritical fluids are exploited for efficient separation and extraction processes.
